In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import *
%matplotlib inline

In [2]:
gene_data=pd.read_csv('gene_high_throughput_sequencing.csv')

In [3]:
gene_data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [17]:
columns_data=np.array(gene_data.columns)
columns_data=np.delete(columns_data,[0,1])

In [20]:
gene_data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [91]:
gene_norm_data=gene_data[gene_data.Diagnosis=='normal']
gene_neoplasia_data=gene_data[gene_data.Diagnosis=='early neoplasia']
gene_cancer_data=gene_data[gene_data.Diagnosis=='cancer']

In [96]:
ttest_p_value_norm=scipy.stats.ttest_ind(gene_norm_data.iloc[:,2:],gene_neoplasia_data.iloc[:,2:],equal_var=False)[1]

In [97]:
ttest_p_value_norm

array([  6.90766016e-01,   3.17853128e-05,   6.02726639e-02, ...,
         6.70394526e-01,   7.93924592e-01,   6.61031158e-01])

In [98]:
ttest_p_value_norm_5=ttest_p_value_norm[ttest_p_value_norm<0.05]

In [100]:
ttest_p_value_norm_5.shape

(1575L,)

In [101]:
ttest_p_value_neoplasia=scipy.stats.ttest_ind(gene_neoplasia_data.iloc[:,2:],
                                             gene_cancer_data.iloc[:,2:],
                                             equal_var=False)[1]

In [103]:
ttest_p_value_neoplasia_5=ttest_p_value_neoplasia[ttest_p_value_neoplasia<0.05]

In [104]:
ttest_p_value_neoplasia_5.shape

(3490L,)

In [105]:
_,ttest_normal_corr,_,_=multipletests(ttest_p_value_norm,alpha=0.05,method='holm')
_,ttest_neoplasia_corr,_,_=multipletests(ttest_p_value_neoplasia,alpha=0.05,method='holm')

In [106]:
print ttest_normal_corr
print ttest_neoplasia_corr

[ 1.          0.50017368  1.         ...,  1.          1.          1.        ]
[ 1.  1.  1. ...,  1.  1.  1.]


In [107]:
p_corr=np.array([ttest_normal_corr,ttest_neoplasia_corr])

In [108]:
p_corr

array([[ 1.        ,  0.50017368,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [111]:
_,p_bonf_corr,_,_=multipletests(p_corr,is_sorted=True,method='bonferroni')

In [112]:
p_bonf_corr

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [113]:
def fold_change(C,T):
    if T>=C:
        return T/C
    else:
        return -C/T

In [138]:
gen_norm_bonf_p_5=gene_norm_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr[0]<0.05)[0]]
gen_neoplasia_bonf_p_5=gene_neoplasia_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr[0]<0.05)[0]]

In [142]:
fc_sum1=0
for norm,neoplasia in zip(gen_norm_bonf_p_5.mean(),gen_neoplasia_bonf_p_5.mean()):
    fc=fold_change(norm,neoplasia)
    if abs(fc)>1.5:
        fc_sum1+=1

In [143]:
fc_sum1

2

In [152]:
gen_neoplasia_bonf_p_5_1=gene_neoplasia_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr[1]<0.05)[0]]
gen_cancer_bonf_p_5=gene_cancer_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr[1]<0.05)[0]]

In [153]:
fc_sum2=0
for neoplasia,cancer in zip(gen_neoplasia_bonf_p_5_1.mean(),gen_cancer_bonf_p_5.mean()):
    fc=fold_change(neoplasia,cancer)
    if abs(fc)>1.5:
        fc_sum2+=1

In [154]:
fc_sum2

77

In [155]:
with open('answer3.txt', 'w') as fout:
    fout.write(str(fc_sum1))
    
with open('answer4.txt', 'w') as fout:
    fout.write(str(fc_sum2))

In [156]:
_,ttest_normal_corr_fdr,_,_=multipletests(ttest_p_value_norm,alpha=0.05,method='fdr_bh')
_,ttest_neoplasia_corr_fdr,_,_=multipletests(ttest_p_value_neoplasia,alpha=0.05,method='fdr_bh')

In [157]:
print ttest_normal_corr_fdr
print ttest_neoplasia_corr_fdr

[ 0.96651052  0.03569758  0.53610276 ...,  0.96312033  0.97735971
  0.96074074]
[ 0.67519508  0.83640641  0.28887272 ...,  0.76659316  0.78132735
  0.82726664]


In [158]:
p_corr_fdr=np.array([ttest_normal_corr_fdr,ttest_neoplasia_corr_fdr])

In [159]:
p_corr_fdr

array([[ 0.96651052,  0.03569758,  0.53610276, ...,  0.96312033,
         0.97735971,  0.96074074],
       [ 0.67519508,  0.83640641,  0.28887272, ...,  0.76659316,
         0.78132735,  0.82726664]])

In [160]:
_,p_bonf_corr_fdr,_,_=multipletests(p_corr_fdr,is_sorted=True,method='bonferroni')

In [90]:
import statsmodels.stats.multitest as smm
from statsmodels.sandbox.stats.multicomp import multipletests

569

In [161]:
p_bonf_corr_fdr

array([[ 1.        ,  0.07139516,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  0.57774543, ...,  1.        ,
         1.        ,  1.        ]])

In [162]:
gen_norm_bonf_p_5_fdr=gene_norm_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr_fdr[0]<0.05)[0]]
gen_neoplasia_bonf_p_5_fdr=gene_neoplasia_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr_fdr[0]<0.05)[0]]

In [163]:
fc_sum1_fdr=0
for norm,neoplasia in zip(gen_norm_bonf_p_5_fdr.mean(),gen_neoplasia_bonf_p_5_fdr.mean()):
    fc=fold_change(norm,neoplasia)
    if abs(fc)>1.5:
        fc_sum1_fdr+=1

In [164]:
fc_sum1_fdr

4

In [165]:
gen_neoplasia_bonf_p_5_1_fdr=gene_neoplasia_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr_fdr[1]<0.05)[0]]
gen_cancer_bonf_p_5_fdr=gene_cancer_data.iloc[:,2:].iloc[:,np.where(p_bonf_corr_fdr[1]<0.05)[0]]

In [166]:
fc_sum2_fdr=0
for neoplasia,cancer in zip(gen_neoplasia_bonf_p_5_1_fdr.mean(),gen_cancer_bonf_p_5_fdr.mean()):
    fc=fold_change(neoplasia,cancer)
    if abs(fc)>1.5:
        fc_sum2_fdr+=1

In [167]:
fc_sum2_fdr

524

In [168]:
with open('answer5.txt', 'w') as fout:
    fout.write(str(fc_sum1_fdr))
    
with open('answer6.txt', 'w') as fout:
    fout.write(str(fc_sum2_fdr))